# Обработаем данные благодоря Embeddings

импортируем класс LocalEmbeddings из [другого проекта(собстеного)](https://github.com/Filip-GG/BreakingGPT/blob/main/localEmbeddings.py)

In [3]:
from LocalEmbeddings import LocalEmbeddings

Embeddings захостим на локальной машине

In [48]:
emded = LocalEmbeddings(
    linck='http://172.20.10.7:1234/v1/embeddings',
    model='nomic-embed-text-v1.5',
    sleep_interval=0
)

Прогоним весь дада-сет через Embeddings

In [7]:
import pandas as pd
import numpy as np

In [9]:
data_train = pd.read_csv('./data_set/train.csv')
data_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Проверим на балансировку (дата сет сбалансированный)

In [12]:
data_train['target'].sum()/(len(data_train['target']))


0.4296597924602653

Функция для пролучения данных

In [30]:
def get_item(index, data_set):
    return str(data_set['text'][index])+' Loc: '+str(data_set['location'][index])

In [32]:
red_data = pd.DataFrame({
    'id':[],
    'text':[],
    'lable':[]
})

In [37]:
for index in range(len(data_train)):
    red_data.loc[index] = {
        'id':data_train['id'][index],
        'text':get_item(index, data_train),
        'lable':data_train['target'][index]
    }

In [38]:
red_data

,id,text,lable
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada Loc: nan,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


Получим вектора

In [49]:
embed_data = emded.embed_documents(red_data['text'])

In [52]:
matr = np.matrix(embed_data)

Получим таблицу данных

In [62]:
embed_datafolder = pd.DataFrame(matr)
embed_datafolder = pd.concat([embed_datafolder, red_data['lable']], axis=1)
embed_datafolder.T

,0,1,2,3,4,5,6,7,8,9,...,7603,7604,7605,7606,7607,7608,7609,7610,7611,7612
0,0.128812,0.035082,-0.010572,-0.002670,0.029154,0.022712,0.047507,0.011506,-0.002745,0.030604,...,0.089176,-0.016607,0.015698,0.036847,0.060509,0.020369,0.050316,-0.022721,0.042684,0.058447
1,0.053218,-0.046052,-0.031801,0.002039,-0.010900,-0.034370,0.007763,-0.024250,0.000451,-0.004539,...,0.011039,0.016594,-0.009033,0.032617,0.050645,0.042225,0.015410,0.024437,0.061103,0.018453
2,-0.141125,-0.200435,-0.174668,-0.195697,-0.194051,-0.205610,-0.204081,-0.195532,-0.190698,-0.167751,...,-0.193321,-0.204413,-0.179652,-0.142232,-0.220891,-0.170382,-0.196051,-0.201502,-0.178112,-0.198673
3,-0.033352,-0.053461,-0.029396,0.004209,-0.044799,0.007703,0.002040,-0.021515,-0.038806,-0.018943,...,-0.041206,-0.043340,0.009144,-0.038434,-0.009723,-0.054964,-0.005353,-0.017330,-0.002306,-0.009448
4,0.085782,-0.000178,0.067843,0.063040,0.033742,0.063994,0.037602,0.049651,0.034051,0.039186,...,0.071708,0.062104,0.047535,0.047538,0.040610,0.033603,0.063749,0.058315,0.083315,0.057897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,0.011844,-0.010960,-0.017383,-0.023751,-0.045394,-0.013544,-0.004591,-0.022570,-0.014997,-0.012800,...,-0.007748,-0.026840,0.001450,-0.048723,-0.033552,-0.026434,-0.041570,-0.009553,-0.023036,0.016407
765,-0.049342,-0.053652,-0.050670,-0.057375,-0.092764,-0.084176,-0.039364,-0.033720,-0.038670,-0.021374,...,-0.069064,-0.037232,-0.057751,-0.074094,-0.046242,-0.026285,-0.046594,-0.038905,-0.051363,-0.078048
766,-0.041732,-0.035593,-0.025131,-0.035191,-0.021682,-0.033005,-0.057694,-0.040795,-0.061152,-0.071435,...,-0.025886,-0.045190,-0.054995,-0.029201,-0.067608,-0.071823,-0.047522,-0.048276,-0.022467,-0.041704
767,-0.069296,-0.023610,-0.075830,-0.058852,-0.031190,-0.044220,-0.044908,0.003318,-0.013899,-0.016136,...,-0.049518,-0.056963,-0.047889,-0.039677,-0.050483,-0.033561,-0.067015,-0.006142,0.025434,-0.081652


сохраним для удобства

In [63]:
embed_datafolder.to_csv('embedd_datafolder.csv')

Также обработаем тестовую 

In [65]:
data_test = pd.read_csv('./data_set/test.csv')
red_data = pd.DataFrame({
    'id':[],
    'text':[]
})
for index in range(len(data_test)):
    red_data.loc[index] = {
        'id':data_test['id'][index],
        'text':get_item(index, data_test)
    }

In [66]:
embed_data_test = emded.embed_documents(red_data['text'])

In [68]:
matr = np.matrix(embed_data_test)
embed_datafolder_test = pd.DataFrame(matr)
embed_datafolder_test = pd.concat([red_data['id'], embed_datafolder_test], axis=1)
embed_datafolder_test.T

,0,1,2,3,4,5,6,7,8,9,...,3253,3254,3255,3256,3257,3258,3259,3260,3261,3262
id,0.000000,2.000000,3.000000,9.000000,11.000000,12.000000,21.000000,22.000000,27.000000,29.000000,...,10838.000000,10845.000000,10856.000000,10857.000000,10858.000000,10861.000000,10865.000000,10868.000000,10874.000000,10875.000000
0,0.038981,0.065005,-0.016430,-0.016252,0.009271,0.074574,-0.002368,0.002031,-0.016987,0.047844,...,0.069338,0.091074,0.001955,-0.042475,0.040721,0.060366,0.066522,0.042555,0.003677,0.010419
1,-0.000382,0.018783,-0.010222,-0.001774,-0.020641,0.051528,-0.015848,-0.040402,0.004356,-0.006087,...,-0.024023,0.013120,0.052941,0.035623,0.001343,0.052060,-0.011654,0.038304,-0.010360,0.021304
2,-0.155328,-0.176865,-0.199719,-0.201118,-0.182101,-0.184662,-0.183198,-0.125755,-0.153329,-0.152064,...,-0.164047,-0.193467,-0.183919,-0.174214,-0.188952,-0.170086,-0.204065,-0.152610,-0.183222,-0.198116
3,-0.044441,0.005965,-0.041369,-0.059935,-0.048837,-0.060705,-0.032146,-0.079671,-0.089232,-0.059780,...,-0.016797,-0.040100,-0.023430,-0.033292,-0.025608,-0.023594,-0.034140,-0.049047,-0.031234,-0.058987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,-0.002177,0.029483,0.028643,0.005867,0.037321,-0.013306,0.003592,0.057639,0.069085,0.065189,...,0.030079,0.038154,0.004760,0.021788,-0.000680,-0.009042,-0.010620,-0.034031,-0.001082,0.035923
764,0.014289,-0.016021,-0.025806,-0.017314,-0.002146,-0.033066,0.004559,-0.000286,0.012229,-0.001828,...,0.011308,0.001962,-0.027219,-0.036279,-0.005103,-0.000036,0.033263,-0.044575,-0.001340,-0.010299
765,-0.022926,-0.054113,-0.044062,-0.060950,0.002449,-0.025492,-0.062674,0.018876,0.006718,-0.016808,...,0.004878,-0.088123,-0.050787,-0.004099,-0.025401,-0.052769,-0.052471,-0.041642,-0.041865,-0.042087
766,-0.036158,-0.046499,-0.041277,-0.055333,-0.042377,-0.062691,-0.033156,-0.046958,-0.046825,-0.044399,...,-0.023648,-0.022209,-0.055904,-0.020943,-0.049334,-0.022533,-0.013297,-0.063953,-0.071988,-0.064540


In [69]:
embed_datafolder_test.to_csv('embedd_datafolder_test.csv')

# Обработаем  полученные данные

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('./embedd_datafolder.csv') 

In [31]:
train_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,lable
0,0,0.128812,0.053218,-0.141125,-0.033352,0.085782,-0.005761,0.009611,0.020561,-0.012852,...,0.011678,0.034116,-0.023189,0.040237,-0.000649,0.011844,-0.049342,-0.041732,-0.069296,1
1,1,0.035082,-0.046052,-0.200435,-0.053461,-0.000178,0.018556,-0.066566,0.015180,-0.044373,...,-0.024148,0.009294,-0.013528,0.012241,0.021835,-0.010960,-0.053652,-0.035593,-0.023610,1
2,2,-0.010572,-0.031801,-0.174668,-0.029396,0.067843,0.027632,-0.030748,0.068039,-0.020873,...,0.020491,0.043029,-0.023328,0.038486,0.026500,-0.017383,-0.050670,-0.025131,-0.075830,1
3,3,-0.002670,0.002039,-0.195697,0.004209,0.063040,-0.005201,-0.052874,0.045588,0.012496,...,0.024993,0.046464,-0.012858,0.027597,0.040484,-0.023751,-0.057375,-0.035191,-0.058852,1
4,4,0.029154,-0.010900,-0.194051,-0.044799,0.033742,-0.000282,-0.015112,0.017055,0.038251,...,0.019028,0.058301,-0.015345,0.022360,0.001924,-0.045394,-0.092764,-0.021682,-0.031190,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,0.020369,0.042225,-0.170382,-0.054964,0.033603,0.095174,-0.037982,0.026633,-0.047131,...,0.029271,0.053022,-0.049020,0.029488,-0.001122,-0.026434,-0.026285,-0.071823,-0.033561,1
7609,7609,0.050316,0.015410,-0.196051,-0.005353,0.063749,0.028375,-0.051032,0.039898,0.017330,...,0.038486,0.047199,-0.039171,-0.014704,0.017697,-0.041570,-0.046594,-0.047522,-0.067015,1
7610,7610,-0.022721,0.024437,-0.201502,-0.017330,0.058315,0.079031,-0.012434,-0.003188,-0.056845,...,-0.016237,0.041771,-0.050770,0.052677,0.018903,-0.009553,-0.038905,-0.048276,-0.006142,1
7611,7611,0.042684,0.061103,-0.178112,-0.002306,0.083315,0.039393,0.012522,-0.008240,-0.039511,...,0.002395,0.041329,-0.032155,0.019770,0.001428,-0.023036,-0.051363,-0.022467,0.025434,1


In [3]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    train_data.to_numpy()[:, 1:-1], 
    train_data.to_numpy()[:, -1],
    train_size= 0.8,
    random_state=42
    )

Обучим модель:

In [35]:
from catboost import CatBoostClassifier
from catboost import Pool

In [36]:
X_train[0].__len__()

768

In [37]:
train = Pool(X_train, y_train)
test = Pool(X_test, y_test)

In [71]:
model = CatBoostClassifier(
    iterations= 1000,
    learning_rate= 0.01,
    eval_metric= 'Accuracy',
    task_type= 'GPU',
    early_stopping_rounds= 20,
    verbose= 50
)

In [72]:
model.fit(train)

0:	learn: 0.7395731	total: 93.4ms	remaining: 1m 33s
50:	learn: 0.7862069	total: 5.36s	remaining: 1m 39s
100:	learn: 0.8068966	total: 10.5s	remaining: 1m 33s
150:	learn: 0.8221675	total: 15.5s	remaining: 1m 27s
200:	learn: 0.8311987	total: 20.1s	remaining: 1m 19s
250:	learn: 0.8394089	total: 25s	remaining: 1m 14s
300:	learn: 0.8472906	total: 29.6s	remaining: 1m 8s
350:	learn: 0.8550082	total: 34.1s	remaining: 1m 3s
400:	learn: 0.8605911	total: 38.7s	remaining: 57.8s
450:	learn: 0.8661741	total: 43.4s	remaining: 52.8s
500:	learn: 0.8725780	total: 48s	remaining: 47.8s
550:	learn: 0.8761905	total: 52.6s	remaining: 42.8s
600:	learn: 0.8806240	total: 57.2s	remaining: 38s
650:	learn: 0.8829228	total: 1m 1s	remaining: 33.1s
700:	learn: 0.8858785	total: 1m 6s	remaining: 28.3s
750:	learn: 0.8901478	total: 1m 11s	remaining: 23.6s
800:	learn: 0.8931034	total: 1m 15s	remaining: 18.8s
850:	learn: 0.8973727	total: 1m 20s	remaining: 14s
900:	learn: 0.9001642	total: 1m 24s	remaining: 9.3s
950:	learn: 0

In [73]:
model.score(test)

0.8266579120157583

Получим sumbit

In [74]:
sumb = pd.read_csv('./embedd_datafolder_test.csv')

In [75]:
out = model.predict(sumb.to_numpy()[:,2:])
out_data = pd.DataFrame({
    'id':sumb['id'],
    'target':list(map(int, out))
})
out_data.to_csv('sumb.csv', index=False)